In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import os
import time
from random import randint

In [2]:
options = Options()
options.add_experimental_option("prefs", {
  "download.default_directory": os.getcwd() + "\Downloads",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})


In [3]:
def findr_scraper(record):
#Instatiate the webdriver and set to implicitly wait for page objects to load
    driver = webdriver.Chrome(options=options, executable_path=r'C:\chromedriver.exe')
    driver.implicitly_wait(5) # seconds loading implicitly

    #Set driver to go to start_page and search a given name
    start_page = "https://1findr.1science.com/search"
    driver.get(start_page)
    search_bar = driver.find_element_by_xpath('//textarea[1]')
    search_bar.send_keys("author:" + "\"" + record.faculty + "\"")
    search_bar.send_keys(Keys.ENTER)
    
    results = False
    if not driver.find_elements_by_xpath('//*[@class="serp-page__no-results"]'):
        results = True

    next_button_exists = False
    if results:
        if driver.find_elements_by_xpath('//*[@class="paginator__next "]'):
            next_button =  driver.find_element_by_xpath('//*[@class="paginator__next "]')
            next_button_exists = True
        citation = open("findr_citations.csv", "a")
        more_results = True
        while more_results:
            cite_buttons = driver.find_elements_by_xpath('//div[@class="search-result"]//button[@class="btn search-result__actions__export"]')
            for cite_button in cite_buttons:
                webdriver.ActionChains(driver).move_to_element(cite_button ).click(cite_button).perform()

                #Click the button to export files
                time.sleep(4)
                export_button = driver.find_element_by_xpath('//div[@class="citationmodal"]//a[@class="nav-link"]')
                webdriver.ActionChains(driver).move_to_element(export_button).click(export_button).perform()

                #select the csv button
                time.sleep(4)
                csv_select = driver.find_element_by_xpath('//input[@value="csv"]')
                webdriver.ActionChains(driver).move_to_element(csv_select).click(csv_select).perform()

                #click the download button
                download_button = driver.find_element_by_xpath('//div[@class="citationmodal"]/div/div/a')
                webdriver.ActionChains(driver).move_to_element(download_button).click(download_button).perform()

                # here I grab the file name; the trick to this is making sure that its the only file in downloads
                time.sleep(4)
                citation_file = os.listdir(os.getcwd() + "\Downloads")[0]

                # grab the citation
                f = open("Downloads\\"+citation_file, errors = 'replace')
                citation_text = f.readlines()[1:][0]
                f.close()
                # delete it
                os.remove("Downloads\\"+citation_file)                
                
                citation_split = citation_text.split(',')
                temp = record.faculty.strip()

                for split in citation_split:
                    temp += "," + split
                temp += "\n"
            
                
                citation.write(temp)

            if not driver.find_elements_by_xpath('//*[@class="paginator__next is-disabled"]'):
                #Click the next button and reselect the next button
                if next_button_exists:
                    next_button =  driver.find_element_by_xpath('//*[@class="paginator__next "]')
                    webdriver.ActionChains(driver).move_to_element(next_button).click(next_button).perform()
                if driver.find_elements_by_xpath('//*[@class="paginator__next "]'):
                    next_button =  driver.find_element_by_xpath('//*[@class="paginator__next "]')
                else:
                    more_results = False
                    
        driver.close()

    else:
        no_results = open("findr_no_results.csv", "a")
        no_results.write(record.faculty.strip() + "," + record.college + "," + record.department + "," + record.url + "\n")
        driver.close()

In [12]:
view_content = pd.read_csv("view_content.csv")
print(view_content.columns)
view_content.head()


Index(['faculty', 'college', 'department', 'url'], dtype='object')


,faculty,college,department,url
0,Sandor Samuels,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...
1,Kurt Saunders,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...
2,Hilary Silvia-Goldberg,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...
3,Melanie Williams,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...
4,Gregory Young,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...


In [5]:
# findr_scraper(view_content.iloc[1])

In [6]:
# for i in range(view_content.shape[0]):
#     findr_scraper(view_content.iloc[i])

In [4]:
field_items = pd.read_csv("field_items.csv")
print(field_items.columns)
field_items

Index(['faculty', 'college', 'department', 'url'], dtype='object')


,faculty,college,department,url
0,Behzad Bavarian,Engineering & Computer Science,Department of Manufacturing Systems Engineeri...,https://www.csun.edu/engineering-computer-sci...
1,Kang Chang,Engineering & Computer Science,Department of Manufacturing Systems Engineeri...,https://www.csun.edu/engineering-computer-sci...
2,Robert D. Conner,Engineering & Computer Science,Department of Manufacturing Systems Engineeri...,https://www.csun.edu/engineering-computer-sci...
3,Ileana Costea,Engineering & Computer Science,Department of Manufacturing Systems Engineeri...,https://www.csun.edu/engineering-computer-sci...
4,Shereazad Jimmy Gandhi,Engineering & Computer Science,Department of Manufacturing Systems Engineeri...,https://www.csun.edu/engineering-computer-sci...
5,Bingbing Li,Engineering & Computer Science,Department of Manufacturing Systems Engineeri...,https://www.csun.edu/engineering-computer-sci...
6,Mark Rajai,Engineering & Computer Science,Department of Manufacturing Systems Engineeri...,https://www.csun.edu/engineering-computer-sci...
7,Ahmad Sarfaraz,Engineering & Computer Science,Department of Manufacturing Systems Engineeri...,https://www.csun.edu/engineering-computer-sci...
8,Maryam Tabibzadeh,Engineering & Computer Science,Department of Manufacturing Systems Engineeri...,https://www.csun.edu/engineering-computer-sci...
9,Durul Ulutan,Engineering & Computer Science,Department of Manufacturing Systems Engineeri...,https://www.csun.edu/engineering-computer-sci...


In [16]:
findr_scraper(field_items.iloc[3])

In [5]:
for i in range(field_items.shape[0]):
    findr_scraper(field_items.iloc[i])

UnicodeEncodeError: 'charmap' codec can't encode character '\ufffd' in position 155: character maps to <undefined>